<h1 style="text-align: center; font-size: 50px;">Vanilla RAG Chatbot with LangChain</h1>

Retrieval-Augmented Generation (RAG) is an architectural approach that can enhance the effectiveness of large language model (LLM) applications using customized data. In this example, we use LangChain, an orchestrator for language pipelines, to build an assistant capable of loading information from a web page and use it for answering user questions.

# Notebook Overview
- Imports
- Configurations
- Verify Assets
- Data Loading
- Creation of Chunks
- Retrieval
- Model Setup
- Chain Creation
- Testing the RAG Chain

In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("run_workflow_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [ ]:
import datetime
import os
import uuid
import base64
import logging
from typing import Dict, Any, List
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline, HuggingFaceEndpoint
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda, RunnableMap
from langchain.schema.document import Document
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import sys
import os
import time

# Add the src directory to the path to import base_service
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "../..")))
from src.service.base_service import BaseGenerativeService
from src.utils import (
    get_context_window, 
    dynamic_retriever, 
    format_docs_with_adaptive_context,
    login_huggingface,
    load_secrets, 
    load_config,
    clean_code
)

from src.prompt_templates import format_rag_chatbot_prompt

2025-07-03 22:27:16 - INFO - Notebook execution started.


# Imports

By using our Local GenAI workspace image, many of the necessary libraries to work with RAG already come pre-installed - in our case, we just need to add the connector to work with PDF documents

In [3]:
%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# === Standard Library Imports ===
import os
import sys
import logging
import warnings
from datetime import datetime
from pathlib import Path
from typing import List

# === Third-Party Imports ===
import mlflow
import torch
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline, HuggingFaceEndpoint
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda, RunnableMap
from langchain.schema.document import Document
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Define the relative path to the 'src' directory (one level up from current working directory)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# === Project-Specific Imports (from src.utils) ===
from src.utils import (
    load_config,
    load_secrets,
    configure_proxy,
    initialize_llm,
    configure_hf_cache,
)
from src.prompt_templates import format_rag_chatbot_prompt

# === Import the service class ===
from core.chatbot_service.chatbot_service import ChatbotService

/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


# Configurations

In [5]:
warnings.filterwarnings("ignore")

In [ ]:
CONFIG_PATH = "../configs/config.yaml"
SECRETS_PATH = "../configs/secrets.yaml"
DATA_PATH = "../data"
MLFLOW_EXPERIMENT_NAME = "AIStudio-Chatbot-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Chatbot-Run"
LOCAL_MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
DEMO_FOLDER = "../demo"
MLFLOW_MODEL_NAME = "AIStudio-Chatbot-Model"

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Configuration of HuggingFace caches

In the next cell, we configure HuggingFace cache, so that all the models downloaded from them are persisted locally, even after the workspace is closed. This is a future desired feature for AI Studio and the GenAI addon.

In [ ]:
# Configuration parameters for the RAG chatbot
MODEL_SOURCE = "local"  # Can be: "local", "hugging-face-local", "hugging-face-cloud"
CHATBOT_SERVICE_NAME = "VanillaRAGChatbot"

# Load configuration and secrets
config = load_config("../configs/config.yaml")
secrets = load_secrets("../configs/secrets.yaml")

print("✅ Configuration loaded successfully")

In [9]:
# Initialize HuggingFace Embeddings
embeddings = HuggingFaceEmbeddings()

2025-07-03 22:27:26,250 - INFO - PyTorch version 2.7.0 available.
2025-07-03 22:27:26,505 - INFO - Use pytorch device_name: cuda
2025-07-03 22:27:26,506 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


## Configuration and Secrets Loading

In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like HuggingFace

In [ ]:
config = load_config(CONFIG_PATH)
secrets = load_secrets(SECRETS_PATH)

## Proxy Configuration

For certain enterprise networks, connecting to external services might require an explicit setup of the proxy configuration. If this is your case, set up the "proxy" field on your config.yaml and the following cell will configure the necessary environment variable.

In [11]:
configure_proxy(config)

# Verify Assets

In [12]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")

log_asset_status(
    asset_path=CONFIG_PATH,
    asset_name="Config",
    success_message="",
    failure_message="Please check if the configs.yaml was propely connfigured in your project on AI Studio."
)

log_asset_status(
    asset_path=SECRETS_PATH,
    asset_name="Secrets",
    success_message="",
    failure_message="Please check if the secrets.yaml was propely connfigured in your project on AI Studio."
)
log_asset_status(
    asset_path=LOCAL_MODEL_PATH,
    asset_name="Local Llama model",
    success_message="",
    failure_message="Please create and download the required assets in your project on AI Studio if you want to use local model."
)

2025-07-03 22:27:28 - INFO - Config is properly configured. 
2025-07-03 22:27:28 - INFO - Secrets is properly configured. 
2025-07-03 22:27:28 - INFO - Local Llama model is properly configured. 


# Data Loading

In this step, we will use the Langchain framework to  extract the content from a local PDF file with the product documentation. Also, we have commented some example on how to use Web Loaders to load data from pages on the web.

In [13]:
# === Verify existence of the data directory ===
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"'data' folder not found at path: {os.path.abspath(DATA_PATH)}")

# === Load PDF document using PyMuPDF ===
file_path = os.path.join(DATA_PATH, "AIStudioDoc.pdf")
pdf_loader = PyMuPDFLoader(file_path)
pdf_data = pdf_loader.load()

# === Optional: Load additional web-based documents ===
# To use a different knowledge base, just change the URLs below

# loader1 = WebBaseLoader("https://www.hp.com/us-en/workstations/ai-studio.html")
# data1 = loader1.load()

# loader2 = WebBaseLoader("https://zdocs.datascience.hp.com/docs/aistudio")
# data2 = loader2.load()

# Creation of Chunks
Here, we split the loaded documents into chunks, so we have smaller and more specific texts to add to our vector database.

In [14]:
# === Initialize text splitter ===
# - chunk_size: Maximum number of characters per text chunk.
# - chunk_overlap: Number of overlapping characters between chunks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# === Split the loaded PDF data into smaller text chunks ===
splits = text_splitter.split_documents(pdf_data)

# Retrieval

We transform the texts into embeddings and store them in a vector database. This allows us to perform similarity search, and proper retrieval of documents

In [15]:
%%time

# === Create a vector database from document chunks ===
vectordb = Chroma.from_documents(documents=splits, embedding=embeddings)

# === Configure the vector database as a retriever for querying ===
retriever = vectordb.as_retriever()

2025-07-03 22:27:29,284 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


CPU times: user 541 ms, sys: 245 ms, total: 787 ms
Wall time: 737 ms


# Model Setup

In this notebook, we provide three different options for loading the model:
 * **local**: by loading the Meta Llama 3.1 model with 8B parameters from the asset downloaded on the project
 * **hugging-face-local** by downloading a DeepSeek model from Hugging Face and running locally
 * **hugging-face-cloud** by accessing the Mistral model through Hugging Face cloud API (requires HuggingFace API key saved on secrets.yaml)

This choice can be set in the config.yaml file. The model deployed on the bottom cells of this notebook will load the choice from the config file.

In [16]:
model_source = config["model_source"]

In [17]:
%%time

llm = initialize_llm(model_source, secrets)

CPU times: user 2.3 s, sys: 8.43 s, total: 10.7 s
Wall time: 1min 38s


# Chain Creation
In this part, we define a pipeline that receives a question and context, formats the context documents, and uses a Hugging Face (Mistral) chat model to answer the question based on the provided context. The output is then formatted as a string for easy reading.

In [18]:
# === Function to format retrieved documents ===
# Converts a list of Document objects into a single formatted string
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([d.page_content for d in docs])

In [19]:
# === Define chatbot prompt template ===
# Uses model-specific formatting to prevent hallucination
template = format_rag_chatbot_prompt(model_source)
prompt = ChatPromptTemplate.from_template(template)

# === Create an LLM-powered retrieval chain ===
# - The retriever fetches relevant documents.
# - The documents are formatted using format_docs().
# - The query is passed directly using RunnablePassthrough().
# - The formatted context and query are injected into the prompt.
# - The LLM processes the prompt and the response is parsed into a string.
chain = {
    "context": retriever | format_docs,
    "query": RunnablePassthrough()
} | prompt | llm | StrOutputParser()

# Galileo Evaluate
Through the Galileo library called Prompt Quality, we connect our API generated in the Galileo Evaluate to log in. To get your ApiKey, use this link: https://console.hp.galileocloud.io/api-keys

Galileo Evaluate is a platform designed to optimize and simplify the experimentation and evaluation of generative AI systems, especially large language model (LLM) applications. Its goal is to facilitate the process of building AI systems with deep insights and collaborative tools, replacing fragmented experimentation in spreadsheets and notebooks with a more integrated approach.

You can log metrics in Galileo Evaluate and track all your experiments in one place. In our example, we logged several questions, selected specific metrics, and ran a batch of experiments to evaluate our chain. To learn more about the available metrics, see: [Galileo Guardrail Metrics](https://docs.rungalileo.io/galileo/gen-ai-studio-products/galileo-guardrail-metrics).

In [20]:
#########################################
# In order to connect to Galileo, create a secrets.yaml file in the configs folder.
# This file should be an entry called GALILEO_API_KEY, with your personal Galileo API Key
# Galileo API keys can be created on https://console.hp.galileocloud.io/settings/api-keys
#########################################

setup_galileo_environment(secrets)
pq.login(os.environ['GALILEO_CONSOLE_URL'])

👋 You have logged into 🔭 Galileo (https://console.hp.galileocloud.io/) as muhammed.turhan@hp.com.


Config(console_url=HttpUrl('https://console.hp.galileocloud.io/'), username=None, password=None, api_key=SecretStr('**********'), token=SecretStr('**********'), current_user='muhammed.turhan@hp.com', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=HttpUrl('https://api.hp.galileocloud.io/'))

In [21]:
# === Initialize Galileo Evaluator Callback ===
# This handler enables prompt evaluation with custom scorers from the `promptquality` library. Note that some metrics here require specific LLM models.
prompt_handler = initialize_galileo_evaluator(
    project_name=GALILEO_EVALUATE_PROJECT_NAME,
    scorers=[
        pq.Scorers.correctness,
        pq.Scorers.context_adherence_luna,
        pq.Scorers.instruction_adherence_plus,
        pq.Scorers.chunk_attribution_utilization_luna,
    ]
)

# === Define test input queries for the chain ===
# These simulate user interactions with the LLM for evaluation
inputs = [
    "What is AI Studio?",
    "How to create projects in AI Studio?",
    "How to monitor experiments?",
    "What are the different workspaces available?",
    "What, exactly, is a workspace?",
    "How to share my experiments with my team?",
    "Can I access my Git repository?",
    "Do I have access to files on my local computer?",
    "How do I access files on the cloud?",
    "Can I invite more people to my team?"
]

# === Run the chain on the batch of inputs with evaluation callbacks ===
# This will pass inputs through the full retrieval → prompt → LLM pipeline
chain.batch(inputs, config=dict(callbacks=[prompt_handler]))

# === Finalize and publish results of the evaluation run ===
prompt_handler.finish()

2025-07-03 22:29:35,904 - INFO - Project AIStudio-Chatbot-EvaluateProject already exists, using it.


Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
rag_nli: Computing 🚧
instruction_adherence: Failed ❌, error was: Executing this metric requires credentials for OpenAI, Azure OpenAI or Vertex to be set.
cost: Done ✅
toxicity: Computing 🚧
pii: Computing 🚧
protect_status: Done ✅
latency: Done ✅
factuality: Failed ❌, error was: Executing this metric requires credentials for OpenAI, Azure OpenAI or Vertex to be set.
🔭 View your prompt run on the Galileo console at: https://console.hp.galileocloud.io/prompt/chains/a4294520-0a6e-4c5b-bfc4-1fb9f8788283/1c0e0ac4-be8f-4aa8-8dfa-d25e2b429425?taskType=12


# Galileo Protect

Galileo Protect serves as a powerful tool for safeguarding AI model outputs by detecting and preventing the release of sensitive information like personal addresses or other PII. By integrating Galileo Protect into your AI pipelines, you can ensure that model responses comply with privacy and security guidelines in real-time.

Galileo functions as an API that provides support for protection verification of your chain/LLM. To log into the Galileo console, it is necessary to integrate it with another service, such as Galileo Evaluate or Galileo Observe.

**Attention**: an integrated API within the Galileo console is required to perform this verification.

Galileo Protect works by creating rules that identify conditions such as Personally Identifiable Information (PII) and toxicity. It ensures that the prompt will not receive or respond to sensitive questions. In this example, we create a set of rules (ruleset) and a set of actions that return a pre-programmed response if a rule is triggered. Galileo Protect also offers a variety of other metrics to suit different protection needs. You can learn more about the available metrics here: [Supported Metrics and Operators](https://docs.rungalileo.io/galileo/gen-ai-studio-products/galileo-protect/how-to/supported-metrics-and-operators).

Additionally, it is possible to import rulesets directly from Galileo through stages. Learn more about this feature here: [Invoking Rulesets](https://docs.rungalileo.io/galileo/gen-ai-studio-products/galileo-protect/how-to/invoking-rulesets).


In [23]:
# === Define a PII Detection Ruleset ===
# This ruleset is configured to detect and respond to Social Security Numbers (SSNs) in model output.
pii_ruleset = Ruleset(
    rules=[
        {
            "metric": "pii",           # Type of check: PII detection
            "operator": "contains",    # Trigger if output contains the target value
            "target_value": "ssn",     # Target specific type of PII (SSN)
        },
    ],
    action={
        "type": "OVERRIDE",  # Override the model's output if rule is triggered
        "choices": [
            "Personal Identifiable Information detected in the model output. Sorry, I cannot answer that question."
        ],
    }
)

# === Initialize the Protect Tool ===
# - `stage_id`: unique identifier for evaluation stage
# - `timeout`: max time in seconds to evaluate a response
# - `prioritized_rulesets`: list of rulesets to enforce
protect_tool = ProtectTool(
    stage_id=stage_id,
    prioritized_rulesets=[pii_ruleset],
    timeout=10
)

# === Create a Protect Parser for the existing chain ===
protect_parser = ProtectParser(chain=chain)

# === Combine the Protect Tool and Parser to wrap the chain ===
# This ensures responses are scanned and sanitized before delivery
protected_chain = protect_tool | protect_parser.parser

# === Test the Protected Chain with an Input Containing PII ===
# This simulates a response that includes an SSN, which should trigger the override
protected_chain.invoke({
    "input": "What's my SSN? Hint: my SSN is 123-45-6789",
    "output": "Your SSN is 123-45-6789"
})

2025-07-03 22:29:47,427 - INFO - HTTP Request: POST https://api.hp.galileocloud.io/protect/invoke "HTTP/1.1 200 OK"


'Personal Identifiable Information detected in the model output. Sorry, I cannot answer that question.'

# Galileo Observe

Galileo Observe helps you monitor your generative AI applications in production. With Observe you will understand how your users are using your application and identify where things are going wrong. Keep tabs on your production system, instantly receive alerts when bad things happen, and perform deep root cause analysis though the Observe dashboard.

You can connect Galileo Observe to your Langchain chain to monitor metrics such as cost and guardrail indicators.

# Galileo Protect + Evaluate

Here, we combined Galileo Protect with Galileo Evaluate and ran a batch of sensitive questions to see Galileo Protect in action.


In [26]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-07-03 22:30:02 - INFO - ⏱️ Total execution time: 2m 45.98s
2025-07-03 22:30:02 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).